<a href="https://colab.research.google.com/github/HummusGuy/COS-470-Project-Part-II-/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.ca

In [3]:
import pandas as pd
import locale
import torch
import re
import transformers
from transformers import AutoTokenizer

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')


if torch.cuda.is_available():
  torch.set_default_device("cuda")
  print("cuda")
else:
  torch.set_default_device("cpu")

def import_tsv(file_path):
    return pd.read_csv(file_path, sep='\t')


# Extract of to 5 terms
def extract_difficult_terms(model, pipeline, tokenizer, category_to_passages):
    passage_to_terms = {}
    print(1)
    for passage in category_to_passages:
        input_text = f'''
                    <s>[INST] <<SYS>>
                    extract up to 5 difficult terms from this passage.
                    do not define them, just list them.
                    <</SYS>>
                    {passage}
                    [/INST]
                    '''
        sequences = pipeline(
        input_text,
    do_sample=True, top_k=1, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id
)
        for seq in sequences:
            result = seq['generated_text']

        # Remove prompt echo
        cleaned_result = result.replace(input_text, "")

        passage_to_terms[passage] = cleaned_result

    return passage_to_terms


# Provide an explanation for the selected difficult terms
def explain_difficult_terms(model, pipeline, tokenizer, passage_to_difficult_terms):
    terms_to_explanations = {}

    for passage, terms in passage_to_difficult_terms.items():
        input_text = f'''
                    <s>[INST] <<SYS>>
                    provide a breif one sentence definition for each of the following terms.
                    <</SYS>>
                    {terms}
                    [/INST]
                    '''
        sequences = pipeline(
        input_text,
        do_sample=True, top_k=1, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id,
        )

        for seq in sequences:
            result = seq['generated_text']

        cleaned_result = result.replace(input_text, "")

        terms_to_explanations[terms] = cleaned_result

    return terms_to_explanations


def main():
    model = "meta-llama/Llama-2-7b-chat-hf"
    tokenizer = AutoTokenizer.from_pretrained(model)
    pipeline = transformers.pipeline(
    "text-generation",
    model=model, torch_dtype=torch.float16, device_map="auto",
    max_new_tokens=7500,)

    # Testing data
    data = import_tsv("task2_train.tsv")
    passage_ids = data["snt_id"].tolist()
    passages = data["source_snt"].tolist()

    passage_to_difficult_terms = extract_difficult_terms(model, pipeline, tokenizer,
                                                         passages)

    terms_to_definitions = explain_difficult_terms(model, pipeline, tokenizer, passage_to_difficult_terms)

    passages_list = []
    terms_list = []
    explanations_list = []


    for passage, terms in passage_to_difficult_terms.items():
        explanation = terms_to_definitions.get(terms, "")
        passages_list.append(passage)
        terms_list.append(terms.strip())
        explanations_list.append(explanation)

    results_df = pd.DataFrame({
        "snt_id": passage_ids,
        "Passage": passages_list,
        "Identified Difficult Terms": terms_list,
        "Explanations": explanations_list
    })


    results_df.to_csv("results.tsv", sep="\t", index=False)
    !mv results.tsv /content/


main()

cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

1


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


mv: 'results.tsv' and '/content/results.tsv' are the same file


In [3]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━